**Note**: this notebook does not work, as the `QuadraticProgram` class cannot handle polynomials with higher orders, which is required in this formulation of the problem.

In [1]:
from collections import OrderedDict

# from qiskit import IBMQ
# IBMQ.load_account()
import numpy as np
from docplex.mp.model import Model
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.aqua import aqua_globals
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver, QAOA
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import RealAmplitudes
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.algorithms import MinimumEigenOptimizer

In [2]:
M = [2, 6, 3]
K = [1, 4, 2]
T = [12, 6, 24]
d = 19


def get_time_matrix(M, T):
    r = []
    for i in M:
        tmp = []
        for j in T:
            tmp.append(j / i)
        r.append(tmp)
    return np.array(r)


def get_cost_matrix(time_matrix, K):
    m = []
    for i in range(len(time_matrix)):
        tmp = []
        for j in time_matrix[i]:
            tmp.append(K[i] * j)
        m.append(tmp)
    return m


time_matrix = np.array(get_time_matrix(M, T))
cost_matrix = np.array(get_cost_matrix(time_matrix, K))

print("Time matrix:\n {}".format(time_matrix))
print("Cost matrix:\n {}".format(cost_matrix))

Time matrix:
 [[ 6.  3. 12.]
 [ 2.  1.  4.]
 [ 4.  2.  8.]]
Cost matrix:
 [[ 6.  3. 12.]
 [ 8.  4. 16.]
 [ 8.  4. 16.]]


In [3]:
def sample_most_likely(state_vector):
    if isinstance(state_vector, (OrderedDict, dict)):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])
        repetitions = int(binary_string[-1][1])
        binary_string = binary_string[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x, repetitions
    return [], 0


optimal_key = "1001100000"


def get_stats_for_result(dict_res):
    optimal = 0
    correct = 0
    incorrect = 0
    correct_config = 0
    incorrect_config = 0

    if optimal_key in dict_res:
        optimal = dict_res[optimal_key]
    for key, val in dict_res.items():
        if is_correct(key):
            correct += val
            correct_config += 1
        else:
            incorrect += val
            incorrect_config += 1

    print('most likely solution: ', sample_most_likely(dict_res))
    print("optimal: ", optimal)
    print("correct solutions: ", correct)
    print("incorrect solutions: ", incorrect)
    print("correct configs: ", correct_config)
    print("incorrect configs: ", incorrect_config)


def is_correct(key):
    reversed_key = key[::-1]
    if solution_vector_correct(reversed_key) and execution_time(reversed_key) <= d:
        return True
    return False


correct_machines = ['00', '01', '10']
machine_to_index = {'00': 0, '01': 1, '10': 2}


def solution_vector_correct(vector):
    task1_machine = vector[0:2]
    task2_machine = vector[2:4]
    task3_machine = vector[4:6]

    if task1_machine in correct_machines and task2_machine in correct_machines and task3_machine in correct_machines:
        return True
    return False


def execution_time(k):
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])

    task1_time = time_matrix[task1_machine, 0]
    task2_time = time_matrix[task2_machine, 1]
    task3_time = time_matrix[task3_machine, 2]

    slack_sum = int(k[6]) * 8 + int(k[7]) * 4 + int(k[8]) * 2 + int(k[9]) * 1

    return task1_time + task2_time + task3_time + slack_sum


In [4]:
def get_quadratic_problem(A, B, C):
    mdl = Model(name='workflow1')
    x = {(i, j) : mdl.binary_var(name='x_{0}_{1}'.format(i, j)) for i in range(0, 3) for j in range(0, 2)}
    x.update({i : mdl.binary_var(name='x_{0}'.format(i)) for i in range(7, 11)})
    
    objective = A * mdl.sum(
          cost_matrix[2, i] * (1 - x[(i, 0)]) * (1 - x[(i, 1)])
        + cost_matrix[1, i] * (1 - x[(i, 0)]) * x[(i, 1)]
        + cost_matrix[0, i] * x[(i, 0)] * (1 - x[(i, 1)])
        for i in range(0, 3)
    )
    
    mdl.add_constraint(B * mdl.sum(x[(i, 0)] * x[(i, 1)] for i in range(0, 3)) == 0, "machine usage")

#     mdl.add_constraint(C * (
#         mdl.sum([
#               time_matrix[2, i] * (1 - x[(i, 0)]) * (1 - x[(i, 1)])
#             + time_matrix[1, i] * (1 - x[(i, 0)]) * x[(i, 1)]
#             + time_matrix[0, i] * x[(i, 0)] * (1 - x[(i, 1)])
#             for i in range(0, 3)
#         ]) 
#         + (8 * x[7] + 4 * x[8] + 2 * x[9] + x[10])
#     ) == C * d, "deadline")
    
    mdl.minimize(objective)

    qp = QuadraticProgram()
    qp.from_docplex(mdl)
    return qp

In [5]:
def perform_classical_optimization(exact, qubitOp):
    exact_result = exact.solve(qubitOp)
    print("\n\nNumPy EigenSolver")
    print(exact_result)
    print(exact_result.samples)


def perform_qaoa(qaoa, qubitOp):
    qaoa_result = qaoa.solve(qubitOp)
    print("\n\nQAOA")
    print(qaoa_result)
    print(qaoa_result.samples)


def perform_vqe(qubitOp, vqe):
    vqe_result = vqe.solve(qubitOp)
    print("\n\nVQE")
    print(vqe_result)
    print(vqe_result.samples)

In [6]:
def do_optimization(qubitOp):
    aqua_globals.random_seed = 10598
    quantum_instance_vqe = QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                           seed_simulator=aqua_globals.random_seed,
                                           seed_transpiler=aqua_globals.random_seed)

    quantum_instance_qaoa = QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                            seed_simulator=aqua_globals.random_seed,
                                            seed_transpiler=aqua_globals.random_seed)

    reps = 2
    max_trials = 1000
    entanglement = 'full'

    spsa = SPSA(maxiter=max_trials)
    ry = RealAmplitudes(qubitOp.get_num_binary_vars(), reps=reps, entanglement=entanglement)
    vqe_mes = VQE(quantum_instance=quantum_instance_vqe, var_form=ry, optimizer=spsa)
    qaoa_mes = QAOA(quantum_instance=quantum_instance_qaoa, initial_point=[0., 0.])
    exact_mes = NumPyMinimumEigensolver()

    vqe = MinimumEigenOptimizer(vqe_mes)  # using VQE
    qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
    exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

    perform_vqe(qubitOp, vqe)
    perform_qaoa(qaoa, qubitOp)
    perform_classical_optimization(exact, qubitOp)

In [7]:
A = 1
B = 20
C = 1

In [8]:
qubitOp = get_quadratic_problem(A, B, C)
do_optimization(qubitOp)

QiskitOptimizationError: 'Incompatible problem: Quadratic constraints are not supported. '